# Specviz notebook/GUI interaction on NIRISS WFSS 1D spectra

In environment jdaviz_redshift from rosteen branch on 01/05/2021

Author: Camilla Pacifici, NIRISS Instrument Scientist

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#plotting and tabling
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5)

params={'legend.fontsize':'18','axes.labelsize':'18',
        'axes.titlesize':'18','xtick.labelsize':'18',
        'ytick.labelsize':'18','lines.linewidth':2,
        'axes.linewidth':2,'animation.html': 'html5',
        'figure.figsize':(8,6)}
plt.rcParams.update(params)
plt.rcParams.update({'figure.max_open_warning': 0})
import matplotlib.patches as patches

#import astropy
import astropy
import astropy.units as u
from astropy.io import fits
from astropy.nddata import StdDevUncertainty

#import specutils
import specutils
from specutils import Spectrum1D, SpectralRegion
from specutils.manipulation import extract_region

#import viztools
import jdaviz
from jdaviz import SpecViz

/Users/cpacifici/miniconda3/envs/jdaviz_redshift/lib/python3.8/site-packages/glue/external/echo/__init__.py:3: UserWarning: glue.external.echo is deprecated, import from echo directly instead
  warnings.warn('glue.external.echo is deprecated, import from echo directly instead')


Check versions

In [3]:
print('astropy:',astropy.__version__)
print('specutils:',specutils.__version__)
print('jdaviz:',jdaviz.__version__)

astropy: 4.2
specutils: 1.1
jdaviz: 0.1.dev1258+g6ee99bc


## 1. Load NIRISS pipeline output
We work with the x1d file which contains all extracted 1D spectra.
I want to do this with three filters! (But I only have F150W for now)

In [5]:
filelink = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/specviz_notebook_gui_interaction/basic_F150W_WFSSR_dit1_x1d.fits'
hdu = fits.open(filelink)

hdu.info()

Filename: /Users/cpacifici/.astropy/cache/download/url/33545d5dbd50d26d5a9838f2d2259699/contents
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     259   ()      
  1  EXTRACT1D     1 BinTableHDU     47   3R x 9C   [D, D, D, D, D, J, D, D, D]   
  2  EXTRACT1D     2 BinTableHDU     47   19R x 9C   [D, D, D, D, D, J, D, D, D]   
  3  EXTRACT1D     3 BinTableHDU     47   78R x 9C   [D, D, D, D, D, J, D, D, D]   
  4  EXTRACT1D     4 BinTableHDU     47   107R x 9C   [D, D, D, D, D, J, D, D, D]   
  5  EXTRACT1D     5 BinTableHDU     47   110R x 9C   [D, D, D, D, D, J, D, D, D]   
  6  EXTRACT1D     6 BinTableHDU     47   107R x 9C   [D, D, D, D, D, J, D, D, D]   
  7  EXTRACT1D     7 BinTableHDU     47   105R x 9C   [D, D, D, D, D, J, D, D, D]   
  8  EXTRACT1D     8 BinTableHDU     47   112R x 9C   [D, D, D, D, D, J, D, D, D]   
  9  EXTRACT1D     9 BinTableHDU     47   107R x 9C   [D, D, D, D, D, J, D, D, D]   
 10  EXTRACT1D    10 BinTable

## 2. Open Specviz and load all the 1D spectra
Parsers are not ready for all instruments, so we load the spectrum manually and we write in a Spectrum1D.

**Developer note:**

We need a parser for NIRISS WFSS x1d.fits files.

In [ ]:
viz = SpecViz()
viz.app

In [ ]:
for i in range (2,19):
    spec_load = hdu[i+1].data

    #invert axis
    wave = spec_load['WAVELENGTH']
    flux = spec_load['FLUX']
    error = spec_load['ERROR']
    invwave = wave[::-1]
    invflux = flux[::-1]
    inverror = error[::-1]

    #mask the parts where the sensitivity in the bandpass is poor
    mask = ((invwave>1.34) & (invwave<1.66))

    spec1d = Spectrum1D(spectral_axis=invwave[mask]*u.um, 
                    flux=invflux[mask]*u.Jy, 
                    uncertainty=StdDevUncertainty(inverror[mask]*u.Jy)) #
    
    viz.load_spectrum(spec1d,"NIRISS 1D {}".format(str(i+1)))

    #print(spec1d)

This is just for quick look to see all the spectra that are in the file. We find our favorite spectrum and work with that.

### I select the emission lines by hand in the GUI to create subset 1

### See what data is used in this specviz istance

In [ ]:
dataout = viz.app.get_data_from_viewer("spectrum-viewer")
spec1d_line = dataout["NIRISS 1D 6"]

In [ ]:
print(spec1d_line)

**Developer note:**

This is a workaround while waiting for a specutils PR to be merged. https://github.com/astropy/specutils/pull/722

### See how the subsets are defined

In [ ]:
region = viz.get_spectral_regions()
region['Subset 1']

## Now I can do the same thing in coding
I can define my own region (cont_region) between arbitrary bounds. I choose 1.35um and 1.42um.

In [ ]:
cont_region = SpectralRegion(1.35*u.um,1.42*u.um)
spec1d_el_hand = extract_region(spec1d_line, cont_region)
print(spec1d_el_hand)

Or I can use the region I defined by hand in the GUI (region['Subset 1']).

In [ ]:
spec1d_el_viz = extract_region(spec1d_line, region['Subset 1'])
print(spec1d_el_viz)

I can load the extracted region I made "by hand" in specviz to check the difference with the one selected in the GUI.

In [ ]:
viz.load_spectrum(spec1d_el_hand, "EL")

### I can plot the spectrum and the subset with matplotlib

In [ ]:
plt.plot(spec1d_line.spectral_axis,spec1d_line.flux)
plt.plot(spec1d_el_viz.spectral_axis,spec1d_el_viz.flux)

plt.xlabel("wavelength ({:latex})".format(spec1d_line.spectral_axis.unit))
plt.ylabel("flux ({:latex})".format(spec1d_line.flux.unit))
plt.title("NIRISS ID 6")
plt.show()


## 4. Use the redshift slider in Specviz to find the redshift

I start by opening a new instance of Specviz.

In [ ]:
viz2 = SpecViz()
viz2.app

And I load just the interesting spectrum (spec1d_line).

In [ ]:
viz2.load_spectrum(spec1d_line,"NIRISS 1D lines")

I can use an available line lists or define my own lines (I know I need Hb4861.3 and the [OIII]4958.9,5006.8 doublet) and play with the redshift slider to match the lines in the line list with the lines in the spectrum.

### Get the redshift out in the Spectrum1D object

In [ ]:
spec1d_redshift_out = viz2.app.get_data_from_viewer("spectrum-viewer")["NIRISS 1D lines"]
spec1d_redshift = Spectrum1D(spec1d_redshift_out, redshift=viz2._redshift)

#spec1d_redshift = Spectrum1D(spectral_axis=spec1d_redshift_out.spectral_axis, 
#                             flux=spec1d_redshift_out.flux,
#                             redshift=viz2._redshift)

In [ ]:
print(spec1d_redshift)

## 5. Use specutils to combine the 3 filters for all spectra

I have not created the simulations in the other two filters yet.